# TUTORIAL : Tensorflow basic computation using single CPU or GPU

## Introduction

The aim of this tutorial is to use `AI TRAINING` product to do a **very simple tensor computation** with the `Tensorflow` library and to compare performances of running it over CPU versus GPU.

## Prerequities

* a Public cloud project
* an `AI-TRAINING` notebook job launched with the `Tensorflow 2` preset image ([documentation available here](https://docs.ovh.com/gb/en/ai-training/start-use-notebooks/))
* the notebook resources should have at least 1 GPU

## In practice

### Step 1: Import Tensorflow library

In [1]:
import tensorflow as tf

### Step 2 : Check that you have GPU(s) available on your notebook

In [2]:
# Get the list of all logical GPU device on your notebook
GPU_DEVICES = tf.config.list_logical_devices('GPU')
# Get the list of all logical CPU device on your notebook
CPU_DEVICES = tf.config.list_logical_devices('CPU')
# Keep only the names of each GPU devices
GPU_DEVICES_NAMES = [x.name for x in GPU_DEVICES]
# Keep only the names of each CPU devices
CPU_DEVICES_NAMES = [x.name for x in CPU_DEVICES]
# The number of GPU devices
GPU_DEVICES_NB = len(GPU_DEVICES)
# The number of CPU devices
CPU_DEVICES_NB = len(CPU_DEVICES)

if GPU_DEVICES_NB == 0:
    raise SystemError('No GPU device found')
else:
    print(f'{GPU_DEVICES_NB} GPU device(s) have been found on your notebook :')

for nb in range(GPU_DEVICES_NB):
    gpu_name = GPU_DEVICES_NAMES[nb]
    print(f'* GPU n°{nb} whose name is "{gpu_name}"')
    
print('')
    
if CPU_DEVICES_NB == 0:
    raise SystemError('No CPU device found')
else:
    print(f'{CPU_DEVICES_NB} CPU device(s) have been found on your notebook :')

for nb in range(CPU_DEVICES_NB):
    cpu_name = CPU_DEVICES_NAMES[nb]
    print(f'* CPU n°{nb} whose name is "{cpu_name}"')

2 GPU device(s) have been found on your notebook :
* GPU n°0 whose name is "/device:GPU:0"
* GPU n°1 whose name is "/device:GPU:1"

1 CPU device(s) have been found on your notebook :
* CPU n°0 whose name is "/device:CPU:0"


### Step 3 : Define the operation to benchmark

Here we choose to define a simple function that multiply 2 random vectors of the given length. This is the function that we are going to benchmark over available devices (GPU and CPU).

In [3]:
def random_multiply(vector_length):
    vector_1 = tf.random.normal(vector_length)
    vector_2 = tf.random.normal(vector_length)
    return vector_1 * vector_2

### Step 4 : Define the function executing the operation on GPU device

In [4]:
def gpu_operation(vector_length):
    # If you have several GPU you can select the one to use by changing the used index of GPU_DEVICES_NAMES
    with tf.device(GPU_DEVICES_NAMES[0]):
        random_multiply(vector_length)

### Step 5 : Define the function executing the operation on CPU device

In [5]:
def cpu_operation(vector_length):
    # If you have several CPU you can select the one to use by changing the used index of GPU_DEVICES_NAMES
    with tf.device(CPU_DEVICES_NAMES[0]):
        random_multiply(vector_length)

### Step6 : Launch the benchmark of each device over several vectors of different lengths

Here we are going to iterate over several lengths of vectors and launch a benchmark both on GPU and CPU to observe on which cases GPU is better.

In [6]:
import timeit

# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu_operation([1])
gpu_operation([1])

for i in range(8):
    vector_length = pow(10, i)
    cpu_time = timeit.timeit(f'cpu_operation([{vector_length}])', number=20, setup="from __main__ import cpu_operation")
    gpu_time = timeit.timeit(f'gpu_operation([{vector_length}])', number=20, setup="from __main__ import gpu_operation")
    print(f'Operations on vector of length {vector_length} are {cpu_time/gpu_time}x faster on GPU than CPU')

Operations on vector of length 1 are 0.704837925731848x faster on GPU than CPU
Operations on vector of length 10 are 0.7908874858684348x faster on GPU than CPU
Operations on vector of length 100 are 0.789991316682575x faster on GPU than CPU
Operations on vector of length 1000 are 1.0697176208473278x faster on GPU than CPU
Operations on vector of length 10000 are 2.9363046057797977x faster on GPU than CPU
Operations on vector of length 100000 are 5.222210204326995x faster on GPU than CPU
Operations on vector of length 1000000 are 24.825055092427608x faster on GPU than CPU
Operations on vector of length 10000000 are 260.8940508924052x faster on GPU than CPU


As we can conclude :
* Computation of scalars (i.e vectors of lenght 1) are more efficient on CPU than GPU
* Computation of vectors are quite as efficient on CPU or GPU when length is between 10 and 1000
* Computation of vectors whose length is higher than 1000 are much more efficient on GPU than CPU

## Going further

* For more information about running computations over GPU with TensorFlow we advise you to follow the [official documentation](https://www.tensorflow.org/guide/gpu)
* **Resource consumption** of your notebook is displayed in a dashboard that you can see. **Just execute the following cells to get the URL corresponding to your notebook** session. The credencials needed to access this dashboard are the same than those used for the current notebook.

In [ ]:
import os

if 'NOTEBOOK_ID' in os.environ:
    VARID = "var-notebook=" + os.environ['NOTEBOOK_ID']
    HOST = os.environ['NOTEBOOK_HOST']
    SUBDOMAIN = "notebook"
else:
    VARID =  "var-job=" + os.environ['JOB_ID']
    HOST = os.environ['JOB_HOST']
    SUBDOMAIN = "job"


print(f'Your resource monitoring dashboard URL is :')
print(f'http://{HOST.replace(SUBDOMAIN, "monitoring")}/d/gpu/job-monitoring?orgId=1&from=now-5m&{VARID}&to=now')